In [10]:
import numpy as np
import pandas as pd

# Настройка генератора случайных чисел для воспроизводимости
np.random.seed(19)

def generate_sample(n=100, diff_strength=0):
    """
    Генерация выборки на n строк.
    diff_strength: сила различия между группами (влияет на корреляцию)
    """
    # 1. Генерируем пол (0 - М, 1 - Ж)
    # Сделаем примерно поровну, но с небольшим перекосом, как бывает в жизни
    sex_binary = np.random.choice(['M', 'F'], size=n, p=[0.45, 0.55])

    # 2. Генерируем количественный признак (0-24 балла)
    # Базовое распределение (нормальное)
    scores = []
    for sex in sex_binary:
        noise = np.random.normal(0, 4) # Случайный разброс
        if sex == 'M':
            # У мужчин база 12 + сдвиг
            val = 12 + diff_strength + noise
        else:
            # У женщин база 12 - сдвиг
            val = 12 - diff_strength + noise

        # Ограничиваем рамками теста (0-24) и округляем
        val = int(max(0, min(24, round(val))))
        scores.append(val)

    return pd.DataFrame({'Sex': sex_binary, 'Score': scores})

# === ГЕНЕРАЦИЯ ДАННЫХ ===

# Вариант 9: Делаем сильное различие (diff=4.0), связь будет сильной
df_var9 = generate_sample(n=100, diff_strength=4.0)

print("Генерация завершена.")
print(f"Вариант 9: {len(df_var9)} строк.")
print("\nПример данных Вариант 4 (первые 5 строк):")

Генерация завершена.
Вариант 9: 100 строк.

Пример данных Вариант 4 (первые 5 строк):


In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2

# =============================================================================
# ДАННЫЕ ДЛЯ ВАРИАНТА 9
# =============================================================================

# Таблица А.10. Добавлен столбец Trait10 (Екзальтованість)
# Значения Trait10 выписаны из последнего столбца скриншота 264
raw_data_v9 = [
    # Sex, Trait1, Trait10
    {'Sex': 'M', 'Trait1': 21, 'Trait10': 12},
    {'Sex': 'M', 'Trait1': 0,  'Trait10': 18},
    {'Sex': 'M', 'Trait1': 12, 'Trait10': 6},
    {'Sex': 'M', 'Trait1': 12, 'Trait10': 24},
    {'Sex': 'M', 'Trait1': 21, 'Trait10': 6},
    {'Sex': 'M', 'Trait1': 12, 'Trait10': 12},
    {'Sex': 'M', 'Trait1': 15, 'Trait10': 18},
    {'Sex': 'M', 'Trait1': 21, 'Trait10': 24},
    {'Sex': 'M', 'Trait1': 24, 'Trait10': 6},
    {'Sex': 'F', 'Trait1': 3,  'Trait10': 24},
    {'Sex': 'F', 'Trait1': 0,  'Trait10': 18},
    {'Sex': 'F', 'Trait1': 21, 'Trait10': 12},
    {'Sex': 'F', 'Trait1': 21, 'Trait10': 18},
    {'Sex': 'F', 'Trait1': 24, 'Trait10': 12},
    {'Sex': 'F', 'Trait1': 6,  'Trait10': 12},
    {'Sex': 'F', 'Trait1': 18, 'Trait10': 18},
    {'Sex': 'F', 'Trait1': 12, 'Trait10': 18},
    {'Sex': 'F', 'Trait1': 21, 'Trait10': 18},
    {'Sex': 'F', 'Trait1': 6,  'Trait10': 24},
    {'Sex': 'F', 'Trait1': 6,  'Trait10': 12},
    {'Sex': 'F', 'Trait1': 9,  'Trait10': 24},
    {'Sex': 'F', 'Trait1': 15, 'Trait10': 18},
    {'Sex': 'F', 'Trait1': 9,  'Trait10': 24},
    {'Sex': 'F', 'Trait1': 18, 'Trait10': 24},
    {'Sex': 'F', 'Trait1': 9,  'Trait10': 6},
    {'Sex': 'F', 'Trait1': 3,  'Trait10': 18},
    {'Sex': 'F', 'Trait1': 24, 'Trait10': 18}
]
df = pd.DataFrame(raw_data_v9)

# Данные для ЗАДАНИЯ 8 (Таблица "Женихи и невесты")
# Вариант 9 -> i = 9
i_val = 9
matrix_base = np.array([
    [18212, 1914, 147, 8, 4],
    [5574, 6677, 1112, 85, 18],
    [498, 2171, 2595, 419, 43],
    [98, 368, 1177, 1280, 308],
    [19, 75, 271, 840, 1701]
])
# Умножаем на номер варианта
n_matrix = matrix_base * i_val

print(f"Данные для ВАРИАНТА {i_val} загружены.")
print(f"Размер выборки для задания 8: {np.sum(n_matrix)}")

Данные для ВАРИАНТА 9 загружены.
Размер выборки для задания 8: 410526


In [2]:
print(f"{'='*20} ЗАДАНИЕ 7: Подготовка (Признаки 1 и 10) {'='*20}")

# 1. Расчет медиан
med_x = df['Trait1'].median()
med_y = df['Trait10'].median() # Теперь используем Trait 10
print(f"Медиана (Признак 1): {med_x}")
print(f"Медиана (Признак 10): {med_y}")

# 2. Формирование таблицы 2x2
# a (X>=med, Y>=med)
# b (X>=med, Y<med)
# c (X<med, Y>=med)
# d (X<med, Y<med)

a = len(df[(df['Trait1'] >= med_x) & (df['Trait10'] >= med_y)])
b = len(df[(df['Trait1'] >= med_x) & (df['Trait10'] < med_y)])
c = len(df[(df['Trait1'] < med_x) & (df['Trait10'] >= med_y)])
d = len(df[(df['Trait1'] < med_x) & (df['Trait10'] < med_y)])

print("\nТаблица сопряженности (a, b, c, d):")
print(f"{a}\t{b}")
print(f"{c}\t{d}")

==================== ЗАДАНИЕ 7: Подготовка (Признаки 1 и 10) ====================
Медиана (Признак 1): 12.0
Медиана (Признак 10): 18.0

Таблица сопряженности (a, b, c, d):
10	7
7	3


In [3]:
print(f"{'='*20} ЗАДАНИЕ 7: Коэффициенты {'='*20}")

numerator = a*d - b*c
denominator_a = a*d + b*c
Ka = numerator / denominator_a

denominator_k = np.sqrt((a+b)*(b+d)*(d+c)*(c+a))
Kk = numerator / denominator_k

print(f"Коэффициент ассоциации Ka = {Ka:.4f}")
print(f"Коэффициент контингенции Kk = {Kk:.4f}")

if abs(Ka) >= 0.5 and abs(Kk) >= 0.3:
    print(">> Вывод: Связь подтверждается.")
else:
    print(">> Вывод: Связь не подтверждается.")

==================== ЗАДАНИЕ 7: Коэффициенты ====================
Коэффициент ассоциации Ka = -0.2405
Коэффициент контингенции Kk = -0.1118
>> Вывод: Связь не подтверждается.


In [4]:
print(f"{'='*20} ЗАДАНИЕ 8: Данные {'='*20}")

n = np.sum(n_matrix)
ni_row = np.sum(n_matrix, axis=1)
nj_col = np.sum(n_matrix, axis=0)
m1, m2 = n_matrix.shape

print(f"Общий объем n = {n}")

==================== ЗАДАНИЕ 8: Данные ====================
Общий объем n = 410526


In [5]:
print(f"{'='*20} ЗАДАНИЕ 8: Хи-квадрат {'='*20}")

sum_term = 0
for i in range(m1):
    for j in range(m2):
        nij = n_matrix[i, j]
        if nij > 0:
            sum_term += (nij**2) / (ni_row[i] * nj_col[j])

chi2_st = n * (sum_term - 1)

df_chi = (m1 - 1) * (m2 - 1)
chi2_crit = chi2.ppf(0.95, df_chi)

print(f"X^2 расчетное = {chi2_st:.2f}")
print(f"X^2 критическое = {chi2_crit:.2f}")

==================== ЗАДАНИЕ 8: Хи-квадрат ====================
X^2 расчетное = 522039.88
X^2 критическое = 26.30


In [6]:
print(f"{'='*20} ЗАДАНИЕ 8: Коэффициент Крамера {'='*20}")

min_m = min(m1 - 1, m2 - 1)
C_val = np.sqrt(chi2_st / (n * min_m))

print(f"Коэффициент Крамера C = {C_val:.4f}")

==================== ЗАДАНИЕ 8: Коэффициент Крамера ====================
Коэффициент Крамера C = 0.5638


In [7]:
print(f"{'='*20} ЗАДАНИЕ 8: Информационная характеристика {'='*20}")

def sum_x_ln_x(arr):
    arr_f = np.array(arr, dtype=float)
    mask = arr_f > 0
    return np.sum(arr_f[mask] * np.log(arr_f[mask]))

term1 = sum_x_ln_x(n_matrix)
term2 = sum_x_ln_x(ni_row)
term3 = sum_x_ln_x(nj_col)
term4 = n * np.log(n)

Y2 = 2 * (term1 - term2 - term3 + term4)

print(f"Y^2 = {Y2:.2f}")

==================== ЗАДАНИЕ 8: Информационная характеристика ====================
Y^2 = 399779.73


In [8]:
print(f"{'='*20} ЗАДАНИЕ 8: Пирсон и Чупров {'='*20}")

phi2 = chi2_st / n

Kp = np.sqrt(phi2 / (1 + phi2))

denom_Ku = np.sqrt((m1 - 1) * (m2 - 1))
Ku = np.sqrt(phi2 / denom_Ku)

print(f"K Пирсона = {Kp:.4f}")
print(f"K Чупрова = {Ku:.4f}")

==================== ЗАДАНИЕ 8: Пирсон и Чупров ====================
K Пирсона = 0.7482
K Чупрова = 0.5638


In [13]:
def calculate_biserial_strict(df, variant_name):
    print(f"{'='*20} ЗАДАНИЕ 9: БИСЕРИАЛЬНЫЙ КОЭФФИЦИЕНТ {'='*20}")
    print(f"\n{'='*10} РАСЧЕТ ДЛЯ {variant_name} {'='*10}")

    # 1. Разделение на группы
    group_M = df[df['Sex'] == 'M']['Score'].values
    group_F = df[df['Sex'] == 'F']['Score'].values

    n1 = len(group_M)
    n2 = len(group_F)
    n_total = n1 + n2

    # 2. Средние значения (Y1_bar, Y2_bar)
    y1_bar = np.mean(group_M)
    y2_bar = np.mean(group_F)

    # Общее среднее (Y_bar)
    y_total_bar = df['Score'].mean()

    print(f"Количество М (n1): {n1}")
    print(f"Количество Ж (n2): {n2}")
    print(f"Всего (n): {n_total}")
    print(f"Среднее М: {y1_bar:.2f}")
    print(f"Среднее Ж: {y2_bar:.2f}")

    # 3. Среднеквадратичное отклонение (Sigma)
    # В примере 3.4 (скриншот 260) формула включает (n - 1) в знаменателе
    # sigma = sqrt( sum((yi - y_avg)^2) / (n - 1) )

    sum_squared_diff = np.sum((df['Score'] - y_total_bar)**2)
    sigma_y = np.sqrt(sum_squared_diff / (n_total - 1))

    print(f"Стандартное отклонение (sigma): {sigma_y:.4f}")

    # 4. Формула бисериального коэффициента
    # rb = ( |y1 - y2| / sigma ) * sqrt( (n1 * n2) / (n * (n - 1)) )
    # Примечание: n * (n - 1) под корнем взято из скриншота 260

    term1 = abs(y1_bar - y2_bar) / sigma_y
    term2 = np.sqrt( (n1 * n2) / (n_total * (n_total - 1)) )

    rb = term1 * term2

    print(f"Бисериальный коэффициент (rb): {rb:.4f}")

    # Вывод
    if rb < 0.3:
        conclusion = "Связь слабая."
    elif rb < 0.7:
        conclusion = "Связь средней силы."
    else:
        conclusion = "Связь сильная."
    print(f"ВЫВОД: {conclusion}")

In [14]:
calculate_biserial_strict(df_var9, "ВАРИАНТА 9")

==================== ЗАДАНИЕ 9: БИСЕРИАЛЬНЫЙ КОЭФФИЦИЕНТ ====================

========== РАСЧЕТ ДЛЯ ВАРИАНТА 9 ==========
Количество М (n1): 47
Количество Ж (n2): 53
Всего (n): 100
Среднее М: 15.04
Среднее Ж: 7.49
Стандартное отклонение (sigma): 5.4881
Бисериальный коэффициент (rb): 0.6902
ВЫВОД: Связь средней силы.
